In [1]:
#インポート
from bs4 import BeautifulSoup as bs
import re
import time
import os
import sys
import json
import pandas as pd
import numpy as np
import pickle
import unicodedata

In [2]:
class DataLoad():
    def __init__(self, inpath,):
        self.inpath = inpath

    def load(self):      
        with open(self.inpath, 'r') as f:
            data = pd.DataFrame().from_dict(json.load(f)).T.reset_index(names="question_number")
            f.close()
        return data

In [108]:
inpath = 'ufo2024/ufo-2024/table_retrieval/train/questions.json'
df_train = DataLoad(inpath,).load()
df_train['flag'] = 'train'
inpath = 'ufo2024/ufo-2024/table_retrieval/test/questions.json'
df_test = DataLoad(inpath,).load()
df_test['flag'] = 'test'
inpath = 'ufo2024/ufo-2024/table_retrieval/valid/questions.json'
df_valid = DataLoad(inpath,).load()
df_valid['flag'] = 'valid'

data = pd.concat([df_train, df_test, df_valid], axis=0)   

In [109]:
import xml.etree.ElementTree as ET
tree = ET.parse('ufo2024/label.xml')
root = tree.getroot()
taxonomy = []
for child in root:
    if 'labelLink' in child.tag:
        for child2 in child:
            if 'loc' not in child2.tag and 'labelArc' not in child2.tag:
                taxonomy.append([child2.attrib['id'], child2.text])

tree = ET.parse('ufo2024/label2.xml')
root = tree.getroot()

for child in root:
    if 'labelLink' in child.tag:
        for child2 in child:
            if 'loc' not in child2.tag and 'labelArc' not in child2.tag:
                taxonomy.append([child2.attrib['id'], child2.text])

tree = ET.parse('ufo2024/label3.xml')
root = tree.getroot()
for child in root:
    if 'labelLink' in child.tag:
        for child2 in child:
            if 'loc' not in child2.tag and 'labelArc' not in child2.tag:
                taxonomy.append([child2.attrib['id'], child2.text])

taxonomy.append(['NonControllingInterestsIFRSMember','非支配持分'])
taxonomy.append(['OutsideDirectorsAuditAndSupervisoryCommitteeMember','社外取締役'])
taxonomy.append(['Ordinary share','社外監査役'])


df = pd.DataFrame(taxonomy, columns = ['en_label', 'ja_label']).drop_duplicates()
df['en_label'] = df['en_label'].apply(lambda x: unicodedata.normalize("NFKC", x).replace('label_', ''))

In [110]:
df.query('"OrdinaryShareMember" in en_label')

,en_label,ja_label
2919,OrdinaryShareMember,普通株式


In [111]:
ja_question = []
for index, row in data.iterrows():
    row['ja_question'] = unicodedata.normalize("NFKC", row['question'])
    en_label = re.findall(r'[a-zA-Z]+', row['question'])
    for label in en_label:
        if label in df['en_label'].values:
            ja_label = df.query('en_label == @label')['ja_label'].values[0]
            row['ja_question'] = row['ja_question'].replace(label, ja_label)
        row['ja_question'] = row['ja_question'].replace('(IFRS)', '')
    ja_question.append(row['ja_question'])
data['ja_question'] = ja_question

In [112]:
def process_query(query):
    #質問文を整形する
    if re.search(r"「.*」",query):
        contents = re.search(r"「(.*)」",query).group(0)[1:-1].split("、")
        return query.replace(re.search(r"「.*」",query).group(0), 'の'.join(contents[::-1]))
data['ja_question'] = data['ja_question'].apply(process_query) 

In [113]:
for index, row in data.iterrows():
    try:
        if re.search(r"^(.*?の)", row['ja_question']):
            text = re.sub(r"^(.*?の)", '', row['ja_question'])
            data.loc[index, 'tmp_question'] = text
    except:
        print(row['ja_question'])
        

In [125]:
#タクソノミの英語ラベルと「」で囲まれた部分の抽出
def extract_words(text):
    words = []
    for word in re.findall(r"「.*」", text):
        words.append(word[1:-1])

    words.extend(re.findall(r'[a-zA-Z]+', text))
    
    return words

data['words'] = data['question'].apply(extract_words)

In [126]:
data.head()

,question_number,question,doc_id,table_id,flag,ja_question,tmp_question,words
0,question_train1,株式会社アインホールディングスの2021年時点における「純資産額、経営指標等」は？,S100RHMW,S100RHMW-0101010-tab2,train,株式会社アインホールディングスの2021年時点における経営指標等の純資産額は?,2022年の非連結又は個別における経営指標等の自己資本利益率は?,[純資産額、経営指標等]
1,question_train2,株式会社アインホールディングスの2023年時点における「純資産額、経営指標等」は？,S100RHMW,S100RHMW-0101010-tab2,train,株式会社アインホールディングスの2023年時点における経営指標等の純資産額は?,2023年時点のNo3大株主における大株主の状況の氏名又は名称は?,[純資産額、経営指標等]
2,question_train6,株式会社アインホールディングスの2022年のNonConsolidatedMemberにおけ...,S100RHMW,S100RHMW-0101010-tab3,train,株式会社アインホールディングスの2022年の非連結又は個別における経営指標等の1株当たり当期...,2023年時点のNo9大株主における大株主の状況の氏名又は名称は?,"[１株当たり当期純利益又は当期純損失（△）、経営指標等, NonConsolidatedMe..."
3,question_train7,株式会社アインホールディングスの2023年時点のOrdinaryShareMemberにおけ...,S100RHMW,S100RHMW-0104010-tab25,train,株式会社アインホールディングスの2023年時点の普通株式における株式の総数等の発行済株式の種類は?,2023年時点の議決権制限株式（自己株式等）における議決権の状況の発行済株式の内容は?,"[種類、発行済株式、株式の総数等, OrdinaryShareMember]"
4,question_train8,株式会社アインホールディングスの2023年時点のOrdinaryShareMemberにおけ...,S100RHMW,S100RHMW-0104010-tab27,train,株式会社アインホールディングスの2023年時点の普通株式における所有株式数-単元未満株式の状...,2023年における監査公認会計士等の監査証明業務に基づく報酬-提出会社は?,"[所有株式数－単元未満株式の状況（株）, OrdinaryShareMember]"


In [127]:
data.to_csv('ufo2024/ufo-2024/table_retrieval/data/dataframe/data.csv', index=False)